In [108]:
import pandas as pd

# Falls noch nicht geladen:2025-10-22-qualification_data
df_priv = pd.read_csv("df_eval_private_2025-11-03.csv", low_memory=False)
df_pub = pd.read_csv("df_eval_public_2025-11-03.csv", low_memory=False)
df_history = pd.read_csv("df_history_2025-11-03.csv", low_memory=False)
df_eval = pd.read_csv("df_IDs_for_eval_2025-11-03.csv", low_memory=False)


# 1) Spalte robust in datetime konvertieren
#dt = pd.to_datetime(df["Auftragsende_SOLL"], errors="coerce")

In [109]:
#df_priv
#df_pub
#df_history
#df_eval

In [110]:
# --- Schritt 1: Zeitspalten konvertieren ---

time_cols = [
    "Auftragseingang",
    "Auftragsende_SOLL",
    "AFO_Start_SOLL",
    "AFO_Ende_SOLL",
    "AFO_Start_IST",
    "AFO_Ende_IST",
    "Auftragsende_IST"
]

# Falls einige Spalten fehlen (z.B. in eval datasets), ignorieren
existing_time_cols = [col for col in time_cols if col in df_history.columns]

for col in existing_time_cols:
    df_history[col] = pd.to_datetime(df_history[col], errors="coerce")


In [111]:
df_history[existing_time_cols].isna().sum()


Auftragseingang          0
Auftragsende_SOLL        0
AFO_Start_SOLL           0
AFO_Ende_SOLL            0
AFO_Start_IST            0
AFO_Ende_IST             0
Auftragsende_IST     58851
dtype: int64

In [114]:
#df_history

In [122]:
cutoff_date = pd.Timestamp("2014-01-02")  # 1.1.2014 + alles davor raus

existing_time_cols = [c for c in time_cols if c in df_history.columns]

# Maske: TRUE = Auftrag enthält alte Zeitstempel
mask_old = df_history[existing_time_cols].lt(cutoff_date).any(axis=1)

# Anzahl der betroffenen Zeilen
print("Anzahl Zeilen mit alten Datumswerten:", mask_old.sum())

# Aufträge entfernen
df_history = df_history.loc[~mask_old].copy()

Anzahl Zeilen mit alten Datumswerten: 302


In [124]:
45946
df_history

,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Kapazität,Auftragsende_IST,AFO_Dauer_IST_Stunde,Auftragsdauer_IST_Tag
1395,351,2,Schwenkzylinder,2014-01-05,2,2014-01-07 12:42:00,1,Info,2014-01-06 07:00:00,2014-01-06 07:01:00,2014-01-06 07:00:00,2014-01-06 07:01:00,NaN,NaN,NaN,2014-01-08 09:17:00,0.02,2.29
1402,358,2,Schwenkzylinder,2014-01-03,1,2014-01-07 13:32:00,1,Info,2014-01-06 07:00:00,2014-01-06 07:01:00,2014-01-06 07:00:00,2014-01-06 07:01:00,NaN,NaN,NaN,2014-09-16 10:30:00,0.02,182.44
1403,359,2,Schwenkzylinder,2014-01-02,2,2014-01-07 13:32:00,1,Info,2014-01-06 07:00:00,2014-01-06 07:01:00,2014-01-06 07:00:00,2014-01-06 07:01:00,NaN,NaN,NaN,2014-01-08 09:22:00,0.02,4.30
1417,373,2,Schwenkzylinder,2014-01-03,2,2014-01-08 07:12:00,1,Info,2014-01-06 07:00:00,2014-01-06 07:01:00,2014-01-06 07:00:00,2014-01-06 07:01:00,NaN,NaN,NaN,2014-01-07 14:21:00,0.02,2.92
1419,375,2,Schwenkzylinder,2014-01-05,3,2014-01-08 07:12:00,1,Info,2014-01-06 07:00:00,2014-01-06 07:01:00,2014-01-06 07:00:00,2014-01-06 07:01:00,NaN,NaN,NaN,2014-01-07 13:50:00,0.02,1.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465940,150313,1,Steuerventilmodul,2024-01-12,2,2024-03-04 07:22:00,31,Nachschweißen,2024-03-01 10:11:00,2024-03-01 14:51:00,2024-03-01 13:11:00,2024-03-01 14:28:00,3,Deckel Maho,8.0,NaT,1.28,NaN
1465941,150091,2,Schwenkzylinder,2023-08-31,1,2024-02-28 14:42:00,32,Mechanisch Bearbeiten,2024-02-28 09:11:00,2024-02-28 10:01:00,2024-03-01 13:22:00,2024-03-01 14:30:00,2,Fronius,17.0,NaT,1.13,NaN
1465942,150354,1,Steuerventilmodul,2024-02-06,1,2024-03-04 10:42:00,21,Schweißen,2024-03-01 10:31:00,2024-03-01 11:21:00,2024-03-01 11:07:00,2024-03-01 12:23:00,2,Fronius,20.0,NaT,1.27,NaN
1465943,150352,1,Steuerventilmodul,2023-10-17,1,2024-03-04 10:42:00,21,Schweißen,2024-03-01 09:51:00,2024-03-01 10:41:00,2024-03-01 11:20:00,2024-03-01 12:37:00,2,Fronius,20.0,NaT,1.28,NaN


In [134]:
# --- Schritt: Eval-IDs aus der History entfernen ---

# Set der IDs, die vorhergesagt werden sollen
eval_ids = set(df_eval["AuftragsID"].unique())

# Maske: Zeilen, die zu Eval-IDs gehören
mask_eval = df_history["AuftragsID"].isin(eval_ids)

# Neue Dateien
df_history_clean = df_history.loc[~mask_eval].copy()   # History ohne Eval IDs
df_history_eval_removed = df_history.loc[mask_eval].copy()  # Nur die Eval-Aufträge

# Sortiert nach ID (absteigend)
df_history_eval_removed = df_history_eval_removed.sort_values(
    by="AuftragsID",
    ascending=False
).reset_index(drop=True)


print("Anzahl gelöschter Eval-Zeilen:", mask_eval.sum())
print("Neue History-Größe:", df_history_clean.shape)
print("Eval-Only-Größe:", df_history_eval_removed.shape)


Anzahl gelöschter Eval-Zeilen: 58828
Neue History-Größe: (1360869, 18)
Eval-Only-Größe: (58828, 18)


In [132]:
df_history_eval_removed

,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Kapazität,Auftragsende_IST,AFO_Dauer_IST_Stunde,Auftragsdauer_IST_Tag
0,150368,3,Daempfungseinheit,2024-01-12,1,2024-03-01 11:02:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,2024-03-01 07:00:00,2024-03-01 07:01:00,NaN,NaN,NaN,NaT,0.02,NaN
1,150368,3,Daempfungseinheit,2024-01-12,1,2024-03-01 11:02:00,100,Transport,2024-03-01 07:01:00,2024-03-01 07:31:00,2024-03-01 07:01:00,2024-03-01 07:31:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
2,150367,3,Daempfungseinheit,2023-12-17,1,2024-03-01 10:12:00,100,Transport,2024-03-01 07:01:00,2024-03-01 07:31:00,2024-03-01 07:01:00,2024-03-01 07:31:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
3,150367,3,Daempfungseinheit,2023-12-17,1,2024-03-01 10:12:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,2024-03-01 07:00:00,2024-03-01 07:01:00,NaN,NaN,NaN,NaT,0.02,NaN
4,150366,3,Daempfungseinheit,2023-09-08,1,2024-03-01 10:12:00,100,Transport,2024-03-01 07:01:00,2024-03-01 07:31:00,2024-03-01 07:01:00,2024-03-01 07:31:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58823,128380,2,Schwenkzylinder,2022-03-27,1,2022-08-25 12:52:00,100,Transport,2022-08-25 08:21:00,2022-08-25 08:51:00,2022-08-25 08:44:00,2022-08-25 09:14:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
58824,128380,2,Schwenkzylinder,2022-03-27,1,2022-08-25 12:52:00,12,Richten,2022-08-25 07:31:00,2022-08-25 08:21:00,2022-08-25 07:31:00,2022-08-25 08:44:00,4,DMG Mori,8.0,NaT,1.22,NaN
58825,128380,2,Schwenkzylinder,2022-03-27,1,2022-08-25 12:52:00,100,Transport,2022-08-25 09:41:00,2022-08-25 10:11:00,2023-05-26 09:16:00,2023-05-26 09:46:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
58826,128380,2,Schwenkzylinder,2022-03-27,1,2022-08-25 12:52:00,100,Transport,2022-08-25 07:01:00,2022-08-25 07:31:00,2022-08-25 07:01:00,2022-08-25 07:31:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN


In [146]:
# Eval Files kombinieren (je eine Zeile pro Auftrag)
df_eval_combined = pd.concat([df_pub, df_priv], axis=0, ignore_index=True)

df_eval_full = df_eval_combined.merge(
    df_history_eval_removed,
    on="AuftragsID",
    how="left",
    suffixes=("_eval", "_hist")
)


In [148]:
df_eval_full = df_history_eval_removed.sort_values(
    by="AuftragsID",
    ascending=False
).reset_index(drop=True)
df_eval_full


,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Kapazität,Auftragsende_IST,AFO_Dauer_IST_Stunde,Auftragsdauer_IST_Tag
0,150368,3,Daempfungseinheit,2024-01-12,1,2024-03-01 11:02:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,2024-03-01 07:00:00,2024-03-01 07:01:00,NaN,NaN,NaN,NaT,0.02,NaN
1,150368,3,Daempfungseinheit,2024-01-12,1,2024-03-01 11:02:00,100,Transport,2024-03-01 07:01:00,2024-03-01 07:31:00,2024-03-01 07:01:00,2024-03-01 07:31:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
2,150367,3,Daempfungseinheit,2023-12-17,1,2024-03-01 10:12:00,100,Transport,2024-03-01 07:01:00,2024-03-01 07:31:00,2024-03-01 07:01:00,2024-03-01 07:31:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
3,150367,3,Daempfungseinheit,2023-12-17,1,2024-03-01 10:12:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,2024-03-01 07:00:00,2024-03-01 07:01:00,NaN,NaN,NaN,NaT,0.02,NaN
4,150366,3,Daempfungseinheit,2023-09-08,1,2024-03-01 10:12:00,100,Transport,2024-03-01 07:01:00,2024-03-01 07:31:00,2024-03-01 07:01:00,2024-03-01 07:31:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58823,128380,2,Schwenkzylinder,2022-03-27,1,2022-08-25 12:52:00,100,Transport,2022-08-25 08:21:00,2022-08-25 08:51:00,2022-08-25 08:44:00,2022-08-25 09:14:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
58824,128380,2,Schwenkzylinder,2022-03-27,1,2022-08-25 12:52:00,12,Richten,2022-08-25 07:31:00,2022-08-25 08:21:00,2022-08-25 07:31:00,2022-08-25 08:44:00,4,DMG Mori,8.0,NaT,1.22,NaN
58825,128380,2,Schwenkzylinder,2022-03-27,1,2022-08-25 12:52:00,100,Transport,2022-08-25 09:41:00,2022-08-25 10:11:00,2023-05-26 09:16:00,2023-05-26 09:46:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN
58826,128380,2,Schwenkzylinder,2022-03-27,1,2022-08-25 12:52:00,100,Transport,2022-08-25 07:01:00,2022-08-25 07:31:00,2022-08-25 07:01:00,2022-08-25 07:31:00,Transport/Rüsten,NaN,NaN,NaT,0.50,NaN


In [150]:
# --- Export der bereinigten History (ohne Eval IDs) ---
df_history_clean.to_csv(
    "df_history_clean.csv",
    index=False,
    encoding="utf-8"
)


# --- Export der kombinierten Eval-Datei (pub + priv + history) ---
df_eval_full.to_csv(
    "df_eval_fulldetails.csv",
    index=False,
    encoding="utf-8"
)

print("Export abgeschlossen.")


Export abgeschlossen.
